In [1]:
import numpy as np
import pandas as pd
import gudhi as gd
import sys
import matplotlib.pyplot as plt
import networkx as nx
from ripser import Rips
import pickle
import os
import persim
from sklearn.manifold import TSNE
from matplotlib.pyplot import figure
from sklearn.preprocessing import normalize

In [2]:
class Graph:
    # Constructor
    def __init__(self, edges, n):
 
        # A list of lists to represent an adjacency list
        self.adjList = [[] for _ in range(n)]
 
        # add edges to the directed graph
        for (source, dest, weight) in edges:
            self.adjList[source].append((dest, weight))
 
 
# Perform DFS on the graph and set the departure time of all
# vertices of the graph
def DFS(graph, v, discovered, departure, time):
    # mark the current node as discovered
    discovered[v] = True
 
    # set arrival time – not needed
    # time = time + 1
 
    # do for every edge (v, u)
    for (u, w) in graph.adjList[v]:
        # if `u` is not yet discovered
        if not discovered[u]:
            time = DFS(graph, u, discovered, departure, time)
 
    # ready to backtrack
    # set departure time of vertex `v`
    departure[time] = v
    time = time + 1
 
    return time
 
def findLongestDistance(graph, source, n):
    # `departure` stores vertex number having its departure
    # time equal to the index of it
    departure = [-1] * n
 
    # to keep track of whether a vertex is discovered or not
    discovered = [False] * n
    time = 0
 
    # perform DFS on all undiscovered vertices
    for i in range(n):
        if not discovered[i]:
            time = DFS(graph, i, discovered, departure, time)
 
    cost = [sys.maxsize] * n
    cost[source] = 0
 
    # Process the vertices in topological order, i.e., in order
    # of their decreasing departure time in DFS
    added = []
    
    for i in reversed(range(n)):
     
        # for each vertex in topological order,
        # relax the cost of its adjacent vertices
        v = departure[i]
        # edge from `v` to `u` having weight `w`
        for (u, w) in graph.adjList[v]:
            w = -w     # make edge weight negative
            # if the distance to destination `u` can be shortened by
            # taking edge (v, u), then update cost to the new lower value
            if cost[v] != sys.maxsize and cost[v] + w < cost[u]:
                cost[u] = cost[v] + w
                
            
    
    
    dist = dict()
    for i in range(n):
        dist[i] = {-cost[i]}
        
    return dist

In [3]:
N_lst = []
L_lst = []

In [4]:
paths = """
perf_400_off_high_GENUS.npy
perf_400_off_medium_GENUS.npy
perf_350_off_high_GENUS.npy
perf_350_off_medium_GENUS.npy
perf_500_on_low_GENUS.npy
perf_300_on_high_GENUS.npy
perf_350_on_high_GENUS.npy
perf_350_on_medium_GENUS.npy
perf_450_on_low_GENUS.npy
perf_300_off_high_GENUS.npy
perf_350_on_low_GENUS.npy
perf_400_on_high_GENUS.npy
perf_300_off_low_GENUS.npy
perf_350_off_low_GENUS.npy
perf_400_off_low_GENUS.npy
perf_450_off_low_GENUS.npy
perf_500_off_low_GENUS.npy
perf_450_on_high_GENUS.npy
perf_450_on_medium_GENUS.npy
perf_500_on_medium_GENUS.npy
perf_500_on_high_GENUS.npy
perf_450_off_medium_GENUS.npy
perf_500_off_high_GENUS.npy
perf_500_off_medium_GENUS.npy
usb_400_on_low_GENUS.npy
usb_300_on_low_GENUS.npy
usb_300_off_low_GENUS.npy
usb_350_off_low_GENUS.npy
usb_500_off_low_GENUS.npy
usb_450_off_low_GENUS.npy
usb_450_on_medium_GENUS.npy
usb_450_on_high_GENUS.npy
usb_500_on_high_GENUS.npy
usb_500_on_medium_GENUS.npy
usb_400_off_high_GENUS.npy
usb_300_off_medium_GENUS.npy
usb_400_off_low_GENUS.npy
usb_300_on_medium_GENUS.npy
usb_300_on_high_GENUS.npy
usb_400_on_high_GENUS.npy
usb_400_on_medium_GENUS.npy
usb_350_on_high_GENUS.npy
usb_450_off_high_GENUS.npy
usb_450_off_medium_GENUS.npy
usb_350_off_medium_GENUS.npy
usb_350_off_high_GENUS.npy
dma_400_on_low_GENUS.npy
dma_500_off_medium_GENUS.npy
dma_400_off_low_GENUS.npy
dma_450_off_medium_GENUS.npy
dma_450_off_high_GENUS.npy
dma_500_on_medium_GENUS.npy
dma_350_off_high_GENUS.npy
dma_350_off_medium_GENUS.npy
dma_450_off_low_GENUS.npy
dma_500_on_low_GENUS.npy
dma_350_off_low_GENUS.npy
dma_300_off_low_GENUS.npy
dma_400_off_high_GENUS.npy
dma_400_off_medium_GENUS.npy
dma_350_on_low_GENUS.npy
dma_350_on_high_GENUS.npy
dma_300_on_medium_GENUS.npy
dma_300_on_high_GENUS.npy
dma_450_on_low_GENUS.npy
dma_300_on_low_GENUS.npy
dma_450_on_high_GENUS.npy
dma_450_on_medium_GENUS.npy
dma_400_on_medium_GENUS.npy
dma_400_on_high_GENUS.npy
jpeg_450_on_low_GENUS.npy
jpeg_450_on_high_GENUS.npy
jpeg_450_on_medium_GENUS.npy
jpeg_300_on_high_GENUS.npy
jpeg_400_off_medium_GENUS.npy
jpeg_450_off_medium_GENUS.npy
jpeg_450_off_high_GENUS.npy
jpeg_500_off_medium_GENUS.npy
jpeg_500_off_high_GENUS.npy
jpeg_450_off_low_GENUS.npy
jpeg_400_off_low_GENUS.npy
jpeg_500_off_low_GENUS.npy
jpeg_500_on_high_GENUS.npy
jpeg_400_on_low_GENUS.npy
jpeg_300_on_low_GENUS.npy
jpeg_500_on_low_GENUS.npy
jpeg_350_off_high_GENUS.npy
jpeg_350_off_medium_GENUS.npy
jpeg_400_on_high_GENUS.npy
jpeg_400_on_medium_GENUS.npy
jpeg_300_off_low_GENUS.npy
jpeg_350_on_low_GENUS.npy
jpeg_350_off_low_GENUS.npy
jpeg_350_on_high_GENUS.npy
jpeg_350_on_medium_GENUS.npy
aes_500_off_high_GENUS.npy
aes_500_off_medium_GENUS.npy
aes_500_off_low_GENUS.npy
aes_450_off_high_GENUS.npy
aes_450_off_medium_GENUS.npy
aes_350_on_low_GENUS.npy
aes_300_off_low_GENUS.npy
aes_300_on_low_GENUS.npy
aes_300_off_high_GENUS.npy
aes_300_off_medium_GENUS.npy
aes_400_on_low_GENUS.npy
aes_450_on_medium_GENUS.npy
aes_500_on_high_GENUS.npy
aes_400_off_medium_GENUS.npy
aes_400_off_high_GENUS.npy
aes_400_on_high_GENUS.npy
aes_400_on_medium_GENUS.npy
aes_350_on_medium_GENUS.npy
aes_500_on_low_GENUS.npy
aes_450_on_low_GENUS.npy
aes_400_off_low_GENUS.npy
aes_300_on_high_GENUS.npy
"""
paths = paths.split()

In [5]:
np.save("paths.npy", paths)

In [6]:
for path in paths:
    p_lst = path.split("_")
    if len(p_lst) == 5 and len(p_lst[1]) == 3 and p_lst[-1] == "GENUS.npy":
        print(path)
        netlist_name = path.split(".")[0]
        col_row = np.load(f"/home/zluo/output/{path}")
        edgelist = []
        for edge in col_row.T:
            new_edge = (edge[0], edge[1])
            edgelist.append(new_edge)

        G = nx.DiGraph(edgelist)
        
        nx.write_gpickle(G, f"/home/zluo/input/{netlist_name}.gpickle")

perf_400_off_high_GENUS.npy
perf_400_off_medium_GENUS.npy
perf_350_off_high_GENUS.npy
perf_350_off_medium_GENUS.npy
perf_500_on_low_GENUS.npy
perf_300_on_high_GENUS.npy
perf_350_on_high_GENUS.npy
perf_350_on_medium_GENUS.npy
perf_450_on_low_GENUS.npy
perf_300_off_high_GENUS.npy
perf_350_on_low_GENUS.npy
perf_400_on_high_GENUS.npy
perf_300_off_low_GENUS.npy
perf_350_off_low_GENUS.npy
perf_400_off_low_GENUS.npy
perf_450_off_low_GENUS.npy
perf_500_off_low_GENUS.npy
perf_450_on_high_GENUS.npy
perf_450_on_medium_GENUS.npy
perf_500_on_medium_GENUS.npy
perf_500_on_high_GENUS.npy
perf_450_off_medium_GENUS.npy
perf_500_off_high_GENUS.npy
perf_500_off_medium_GENUS.npy
usb_400_on_low_GENUS.npy
usb_300_on_low_GENUS.npy
usb_300_off_low_GENUS.npy
usb_350_off_low_GENUS.npy
usb_500_off_low_GENUS.npy
usb_450_off_low_GENUS.npy
usb_450_on_medium_GENUS.npy
usb_450_on_high_GENUS.npy
usb_500_on_high_GENUS.npy
usb_500_on_medium_GENUS.npy
usb_400_off_high_GENUS.npy
usb_300_off_medium_GENUS.npy
usb_400_off_low

In [5]:
for path in paths:
    netlist_name = path.split(".")[0]
    print(netlist_name)
    G = nx.read_gpickle(f"/home/zluo/input/{netlist_name}.gpickle")

    p_input = set()
    p_output = set()
    o_d = G.out_degree
    i_d = G.in_degree

    for node in G:
        if o_d[node] == 0 and i_d[node] == 0:
            continue

        if o_d[node] == 0:
            p_output.add(node)

        if i_d[node] == 0:
            p_input.add(node)


    all_raw = []

    for v in p_input:
        all_raw.append(nx.shortest_path_length(G, v))    

    delay_dict = all_raw[0]
    delay_dict = {key: [delay_dict[key]] for key in delay_dict.keys()}
    for dist in all_raw[1:]:
        for key in dist.keys(): 
            val = dist[key]
            if key in delay_dict.keys():
                delay_dict[key] = delay_dict[key] + [val]
            else:
                delay_dict[key] = [val]

    delay_dict = {key: min(delay_dict[key]) for key in delay_dict.keys()}

    with open(f'/home/zluo/output/delay_dict_{netlist_name}_shortest.pkl', 'wb') as f:
        pickle.dump(delay_dict, f)

perf_400_off_high_GENUS
perf_400_off_medium_GENUS
perf_350_off_high_GENUS
perf_350_off_medium_GENUS
perf_500_on_low_GENUS
perf_300_on_high_GENUS
perf_350_on_high_GENUS
perf_350_on_medium_GENUS
perf_450_on_low_GENUS
perf_300_off_high_GENUS
perf_350_on_low_GENUS
perf_400_on_high_GENUS
perf_300_off_low_GENUS
perf_350_off_low_GENUS
perf_400_off_low_GENUS
perf_450_off_low_GENUS
perf_500_off_low_GENUS
perf_450_on_high_GENUS
perf_450_on_medium_GENUS
perf_500_on_medium_GENUS
perf_500_on_high_GENUS
perf_450_off_medium_GENUS
perf_500_off_high_GENUS
perf_500_off_medium_GENUS
usb_400_on_low_GENUS
usb_300_on_low_GENUS
usb_300_off_low_GENUS
usb_350_off_low_GENUS
usb_500_off_low_GENUS
usb_450_off_low_GENUS
usb_450_on_medium_GENUS
usb_450_on_high_GENUS
usb_500_on_high_GENUS
usb_500_on_medium_GENUS
usb_400_off_high_GENUS
usb_300_off_medium_GENUS
usb_400_off_low_GENUS
usb_300_on_medium_GENUS
usb_300_on_high_GENUS
usb_400_on_high_GENUS
usb_400_on_medium_GENUS
usb_350_on_high_GENUS
usb_450_off_high_GENUS


In [53]:
new_all_lst = []

for path in paths:
    netlist_name = path.split(".")[0]
    print(netlist_name)
    G = nx.read_gpickle(f"/home/zluo/input/{netlist_name}.gpickle")
    p_input = set()
    p_output = set()
    o_d = G.out_degree
    i_d = G.in_degree

    for node in G:
        if o_d[node] == 0 and i_d[node] == 0:
            continue

        if o_d[node] == 0:
            p_output.add(node)

        if i_d[node] == 0:
            p_input.add(node)
            
    i_d = [val[-1] for val in list(i_d)]
    o_d = [val[-1] for val in list(o_d)]
    
    pipo = [len(p_input), len(p_output), G.number_of_nodes(), G.number_of_edges()]
    in_degrees = list(np.percentile(i_d, [idx*10 for idx in range(0, 11)]))
    out_degrees = list(np.percentile(o_d, [idx*10 for idx in range(0, 11)]))
    
    with open(f'/home/zluo/output/delay_dict_{netlist_name}_shortest.pkl', 'rb') as f:
        delay_dict_s = pickle.load(f)
        
    with open(f'/home/zluo/output/delay_dict_{netlist_name}.pkl', 'rb') as f:
        delay_dict = pickle.load(f)
    
    l1 = list(delay_dict_s.values())
    l2 = list(delay_dict.values())
    
    l1 = [val for val in l1 if val > 0]
    l2 = [val for val in l2 if val > 0]
    
    dis_lst = list(np.percentile(l2, [idx*10 for idx in range(0, 11)]))
    s_dis_lst = list(np.percentile(l1, [idx*10 for idx in range(0, 11)]))
    mean_sd = [np.mean(i_d), np.std(i_d), np.mean(o_d), np.std(o_d), np.mean(dis_lst), np.std(dis_lst), np.mean(s_dis_lst), np.std(s_dis_lst)]
    new_all_lst.append(pipo+mean_sd+in_degrees+out_degrees+dis_lst+s_dis_lst)

perf_400_off_high_GENUS
perf_400_off_medium_GENUS
perf_350_off_high_GENUS
perf_350_off_medium_GENUS
perf_500_on_low_GENUS
perf_300_on_high_GENUS
perf_350_on_high_GENUS
perf_350_on_medium_GENUS
perf_450_on_low_GENUS
perf_300_off_high_GENUS
perf_350_on_low_GENUS
perf_400_on_high_GENUS
perf_300_off_low_GENUS
perf_350_off_low_GENUS
perf_400_off_low_GENUS
perf_450_off_low_GENUS
perf_500_off_low_GENUS
perf_450_on_high_GENUS
perf_450_on_medium_GENUS
perf_500_on_medium_GENUS
perf_500_on_high_GENUS
perf_450_off_medium_GENUS
perf_500_off_high_GENUS
perf_500_off_medium_GENUS
usb_400_on_low_GENUS
usb_300_on_low_GENUS
usb_300_off_low_GENUS
usb_350_off_low_GENUS
usb_500_off_low_GENUS
usb_450_off_low_GENUS
usb_450_on_medium_GENUS
usb_450_on_high_GENUS
usb_500_on_high_GENUS
usb_500_on_medium_GENUS
usb_400_off_high_GENUS
usb_300_off_medium_GENUS
usb_400_off_low_GENUS
usb_300_on_medium_GENUS
usb_300_on_high_GENUS
usb_400_on_high_GENUS
usb_400_on_medium_GENUS
usb_350_on_high_GENUS
usb_450_off_high_GENUS


In [13]:
dis_lst = []
s_dis_lst = []


for path in paths:
    netlist_name = path.split(".")[0]
    print(netlist_name)
    with open(f'/home/zluo/output/delay_dict_{netlist_name}_shortest.pkl', 'rb') as f:
        delay_dict_s = pickle.load(f)
        
    with open(f'/home/zluo/output/delay_dict_{netlist_name}.pkl', 'rb') as f:
        delay_dict = pickle.load(f)
    
    l1 = list(delay_dict_s.values())
    l2 = list(delay_dict.values())
    
    l1 = [val for val in l1 if val > 0]
    l2 = [val for val in l2 if val > 0]
    
    dis_lst.append(np.percentile(l2, [idx*10 for idx in range(0, 11)]))
    s_dis_lst.append(ls
                    )

perf_400_off_high_GENUS
perf_400_off_medium_GENUS
perf_350_off_high_GENUS
perf_350_off_medium_GENUS
perf_500_on_low_GENUS
perf_300_on_high_GENUS
perf_350_on_high_GENUS
perf_350_on_medium_GENUS
perf_450_on_low_GENUS
perf_300_off_high_GENUS
perf_350_on_low_GENUS
perf_400_on_high_GENUS
perf_300_off_low_GENUS
perf_350_off_low_GENUS
perf_400_off_low_GENUS
perf_450_off_low_GENUS
perf_500_off_low_GENUS
perf_450_on_high_GENUS
perf_450_on_medium_GENUS
perf_500_on_medium_GENUS
perf_500_on_high_GENUS
perf_450_off_medium_GENUS
perf_500_off_high_GENUS
perf_500_off_medium_GENUS
usb_400_on_low_GENUS
usb_300_on_low_GENUS
usb_300_off_low_GENUS
usb_350_off_low_GENUS
usb_500_off_low_GENUS
usb_450_off_low_GENUS
usb_450_on_medium_GENUS
usb_450_on_high_GENUS
usb_500_on_high_GENUS
usb_500_on_medium_GENUS
usb_400_off_high_GENUS
usb_300_off_medium_GENUS
usb_400_off_low_GENUS
usb_300_on_medium_GENUS
usb_300_on_high_GENUS
usb_400_on_high_GENUS
usb_400_on_medium_GENUS
usb_350_on_high_GENUS
usb_450_off_high_GENUS


In [16]:
all_lst = np.load("new_X.npy")
all_lst[0][:-4]

array([5.00650000e+04, 2.39320000e+04, 2.09196891e+00, 1.34162485e+00,
       2.09196891e+00, 1.39352694e+01])

In [21]:
all_lst = np.load("new_X.npy")
new_all_lst = []
for idx in range(len(all_lst)):
    lst = all_lst[idx][:-4]
    lst = lst.tolist()
    lst += list(dis_lst[idx])
    lst += list(s_dis_lst[idx])
    new_all_lst.append(lst)

In [22]:
np.array(new_all_lst).shape

(117, 28)

In [54]:
np.save("new_X.npy", new_all_lst)

In [22]:
z_lst = []
o_lst = []
for path in paths:
    dgms = np.load(f"/home/zluo/new_chip_design/net_dgms/{path}", allow_pickle=True)
    z_lst.append(len(dgms[0]+dgms[1]+dgms[2]))
    o_lst.append(len(dgms[3]))

In [27]:
np.save("o_lst.npy", o_lst)

In [3]:
for path in os.listdir("/home/zluo/output/"):
    p_lst = path.split("_")
    if len(p_lst) == 5 and len(p_lst[1]) == 3 and p_lst[-1] == "GENUS.npy":
        print(path)
        col_row = np.load(f"/home/zluo/output/{path}")
        edgelist = []
        long = True
        col = col_row[0]
        row = col_row[1]
        edgs = np.array([col, row]).T
        
        p_input = []
        p_output = []


        for node in col:
            if node not in row:
                p_input.append(node)

        for node in row:
            if node not in col:
                p_output.append(node)

        p_input = np.unique(np.array(p_input))
        p_output = np.unique(np.array(p_output))
        
        #####################################
        all_dist_raw = []
        edgs_d = [(lst[0], lst[1], 1) for lst in edgs] 
        n = np.max(list(col) + list(row)) + 1
        graph = Graph(edgs_d, n)
        for node in p_input: all_dist_raw.append(findLongestDistance(graph, node, n))

        # building the delay based dictionary
        delay_dict = all_dist_raw[0]
        delay_dict = {key: list(delay_dict[key]) for key in delay_dict.keys()}
        for dist in all_dist_raw[1:]:
            for key in dist.keys(): 
                val = dist[key]
                if key in delay_dict.keys():
                    delay_dict[key] = delay_dict[key] + list(val)
                else:
                    delay_dict[key] = list(val)

        delay_dict = {key: max(delay_dict[key]) for key in delay_dict.keys()}

        with open(f'/home/zluo/output/delay_dict_{path[:-4]}.pkl', 'wb') as f:
            pickle.dump(delay_dict, f)
                
                
        d_lst = []
        for delay in range(max(delay_dict.values()) + 1):
            delay_level_lst = []
            for node in delay_dict.keys():
                if delay_dict[node] == delay:
                    delay_level_lst.append(node)

            if len(delay_level_lst) == 0:
                continue
            d_lst.append(delay_level_lst)
        to_level = len(d_lst) 
        st = gd.SimplexTree()

        for edg in edgs:
            st.insert(edg)

        for delay in range(to_level):
            nodes = d_lst[delay]
            for node in nodes:
                st.assign_filtration([node], delay)

        st.extend_filtration()
        dgms = st.extended_persistence()
        
        np.save(f"/home/zluo/new_chip_design/net_dgms/{path[:-4]}.npy", dgms)

usb_450_off_medium_GENUS.npy
usb_500_on_medium_GENUS.npy


/home/zluo/anaconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


usb_300_on_medium_GENUS.npy
dma_450_off_low_GENUS.npy
aes_500_off_high_GENUS.npy


KeyboardInterrupt: 

In [ ]:
to_search = "/home/zluo/new_chip_design/net_dgms/"
w_d_lst = []
for file in os.listdir(to_search):
    path = to_search + file
    print(path)
    dgm1 = np.load(path, allow_pickle=True)[-1]
    dgm1 = np.array([tp[-1] for tp in dgm1])
    inner_lst = []
    
    for file in os.listdir(to_search):
        dgm2 = np.load(path, allow_pickle=True)[-1]
        dgm2 = np.array([tp[-1] for tp in dgm2])
        inner_lst.append(persim.sliced_wasserstein(dgm1, dgm2, M=50))
        
    w_d_lst.append(inner_lst)
    
np.save("/home/zluo/new_chip_design/w_d_lst_1_nets.npy", w_d_lst)

/home/zluo/new_chip_design/net_dgms/aes_400_off_medium_GENUS.npy
/home/zluo/new_chip_design/net_dgms/dma_450_off_medium_GENUS.npy
/home/zluo/new_chip_design/net_dgms/dma_350_on_low_GENUS.npy
/home/zluo/new_chip_design/net_dgms/perf_300_off_high_GENUS.npy
/home/zluo/new_chip_design/net_dgms/usb_400_on_medium_GENUS.npy
/home/zluo/new_chip_design/net_dgms/jpeg_300_on_high_GENUS.npy
/home/zluo/new_chip_design/net_dgms/aes_300_on_low_GENUS.npy
/home/zluo/new_chip_design/net_dgms/aes_500_off_high_GENUS.npy
/home/zluo/new_chip_design/net_dgms/perf_400_on_high_GENUS.npy
/home/zluo/new_chip_design/net_dgms/jpeg_350_off_high_GENUS.npy


In [1]:
len(w_d_lst)

NameError: name 'w_d_lst' is not defined